In [1]:
import time
import os
import glob
import fitz  # PyMuPDF
import base64
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager

# Set up Selenium WebDriver
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
prefs = {"printing.print_preview_sticky_settings.appState": '{"recentDestinations":[{"id":"Save as PDF"}]}'}
options.add_experimental_option("prefs", prefs)
options.add_argument("--kiosk-printing")  # Auto confirm print
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Load the webpage
driver.get("https://www.coursera.org/?authMode=login")  # Change to your target URL

In [2]:
driver.switch_to.window(driver.window_handles[1])

In [3]:
driver.execute_script("document.body.style.zoom='90%'")

In [6]:
driver.execute_script("return document.getElementsByClassName('rc-TunnelVisionHeader _11sxso0')[0].remove();")

In [7]:
driver.execute_script("return document.getElementsByClassName('cds-AccordionRoot-focusContainer')[0].remove();")

In [8]:
def save_pdf(driver, file_name):
    params = {'landscape': False, 'paperWidth': 12, 'paperHeight': 25}
    data = driver.execute_cdp_cmd("Page.printToPDF", params)
    with open(file_name, 'wb') as file:
        file.write(base64.b64decode(data['data']))
# Usage
output_file = "practical_quiz_1.pdf"
save_pdf(driver, output_file)

In [3]:
import fitz  # PyMuPDF

src = fitz.open("course_1/week_1/practical_quiz_1.pdf")  # Open source PDF
doc = fitz.open()  # Create a new PDF document

for ipage in src:
    rect = ipage.rect  # Get original page dimensions
    crop_rect = fitz.Rect(rect.x0 + 30, rect.y0, rect.x1 - 30, rect.y1)  # Adjust left/right padding

    if rect.width > rect.height:
        fmt = fitz.paper_size("a4-l")  # Landscape A4
    else:
        fmt = fitz.paper_size("a4")  # Portrait A4
    
    page = doc.new_page(width=fmt[0], height=fmt[1])  # Create new page
    page.show_pdf_page(page.rect, src, ipage.number, clip=crop_rect)  # Apply cropping

doc.save("course_1/week_1/practical_quiz_1_reformatted.pdf")
doc.close()